In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import *
from keras.applications import VGG16
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.utils import to_categorical
from keras.callbacks import *
from keras.optimizers import *
from keras.models import *
from keras.layers import *
from keras.utils import to_categorical
from keras.callbacks import *

import os

import PIL

Using TensorFlow backend.


In [2]:
def read_and_resize_single_img(filename,nrow=224,ncol=224,channels=3):
    from scipy import misc
    arr = misc.imread(filename)
    arr = misc.imresize(arr,size=(nrow,ncol,channels))
    return arr

def read_train_from_dirs(path = '../input/train',num_imgs_per_class=5,nrow=224,ncol=224,channels=3):
    import glob
    filelist = glob.glob(path + '*/*.jpg')
    categories = [l.split('\\')[-2] for l in filelist]
    v2k = {l:i for i,l in enumerate(np.unique(categories))}
    l = [[t,v2k[t]] for t in categories]
    labels_df = pd.DataFrame(l,columns=['category','cat_num'])
    labels_df['filenames'] = filelist
    labels_indexes = np.array([np.where(labels_df.cat_num==c)[0][0:num_imgs_per_class] 
                               for c in labels_df.cat_num.unique()[:20]])
    filtered = labels_df.iloc[labels_indexes.flatten(),:]
    num_imgs = filtered.shape[0]
    x = np.zeros((num_imgs,nrow,ncol,channels))
    for i in range(filtered.shape[0]):
        arr=read_and_resize_single_img(filtered.loc[:,'filenames'].iloc[i],nrow,ncol,channels)
        if len(arr.shape)<3: arr = np.stack([arr,arr,arr],axis=-1) #change to use np.repeat() for more generalizable
        x[i] = np.float16(arr)
    y = filtered.loc[:,'cat_num'].values
    return x,y

In [3]:
def plot_multiple_imgs(data,nrows=4,ncols=4,figsize=(18,18)):
    fig,ax = plt.subplots(nrows=nrows,ncols=ncols,figsize=figsize)
    for i in range(nrows*ncols):
        ax[i%nrows,i//nrows].imshow(data[i])

In [4]:
%%time
num_train_imgs=10222
num_test_imgs = 10357
x_train,y_train = read_train_from_dirs(num_imgs_per_class=num_train_imgs)
x_test,y_test = read_train_from_dirs(path='../input/test',num_imgs_per_class=num_test_imgs)

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.


Wall time: 5min 37s


In [ ]:
plot_multiple_imgs(X_train/255,6,6)

In [5]:
def get_model_all_classes():
    vgg16 = VGG16(weights='imagenet', include_top=True)
    vgg16.summary()
    for l in vgg16.layers:
        l.trainable = False

    x = GlobalAveragePooling2D()(vgg16.layers[-5].output)
    x = Dropout(0.3)(x)
#     x = Flatten()(x)
    x = Dense(to_categorical(range(10)).shape[1],activation='softmax',name = 'predictions')(x)
    model = Model(inputs = vgg16.input,outputs = x)
    model.summary()
    return model

In [ ]:
model = get_model_all_classes()
model.compile(loss='categorical_crossentropy',optimizer='adam')

 21241856/553467096 [>.............................] - ETA: 21:44